# Without Grouping

In [80]:
# import pandas as pd

# # Load data for the left sensor
# df_left = pd.read_csv('2ia-2024-hackathon/left_accs.csv', names=['time', 'xl', 'yl', 'zl'])
# df_left['time'] = pd.to_datetime(df_left['time'], unit='s')

# # Load data for the right sensor
# df_right = pd.read_csv('2ia-2024-hackathon/right_accs.csv', names=['time', 'xr', 'yr', 'zr'])
# df_right['time'] = pd.to_datetime(df_right['time'], unit='s')

# # Set `time` as index for both DataFrames
# df_left.set_index('time', inplace=True)
# df_right.set_index('time', inplace=True)

# # Reindex df_left on the timestamps of df_right
# df_left_reindexed = df_left.reindex(df_right.index)

# # Interpolate the `xl`, `yl`, `zl` columns of df_left
# df_left_interpolated = df_left_reindexed.interpolate(method='time')

# df = pd.merge(df_left_interpolated, df_right, left_index=True, right_index=True)
# columns_to_convert = ['xl', 'yl', 'zl', 'xr', 'yr', 'zr']
# for col in columns_to_convert:
#     df[col] = df[col].astype('float16')

# df.reset_index(inplace=True)

# dates_to_remove = [pd.Timestamp('2024-07-19').date(), pd.Timestamp('2024-07-26').date(), pd.Timestamp('2024-07-27').date()]
# df = df[~df['time'].dt.date.isin(dates_to_remove)]

# df['activity'] = 'none'

# df.to_csv('data/data.csv', index=False)

# With Grouping

In [101]:
import pandas as pd

# Load data for the left sensor
df_left = pd.read_csv('2ia-2024-hackathon/left_accs.csv', names=['time', 'xl', 'yl', 'zl'])
df_left['time'] = pd.to_datetime(df_left['time'], unit='s')

# Load data for the right sensor
df_right = pd.read_csv('2ia-2024-hackathon/right_accs.csv', names=['time', 'xr', 'yr', 'zr'])
df_right['time'] = pd.to_datetime(df_right['time'], unit='s')

# Set `time` as index for both DataFrames
df_left.set_index('time', inplace=True)
df_right.set_index('time', inplace=True)

# Reindex df_left on the timestamps of df_right
df_left_reindexed = df_left.reindex(df_right.index)

# Interpolate the `xl`, `yl`, `zl` columns of df_left
df_left_interpolated = df_left_reindexed.interpolate(method='time')

df_right.reset_index(inplace=True)
df_left_interpolated.reset_index(inplace=True)

df = pd.merge(df_left_interpolated, df_right, on='time')
columns_to_convert = ['xl', 'yl', 'zl', 'xr', 'yr', 'zr']
for col in columns_to_convert:
    df[col] = df[col].astype('float16')

dates_to_remove = [pd.Timestamp('2024-07-19').date(), pd.Timestamp('2024-07-26').date(), pd.Timestamp('2024-07-27').date()]
df = df[~df['time'].dt.date.isin(dates_to_remove)]

df['time'] = df['time'].dt.floor('S')
df = df.groupby('time').mean().reset_index()

df['activity'] = 'none'

df.to_csv('data/data_s.csv', index=False)

---

# Activities to data

In [7]:
data = pd.read_csv('data/data_s.csv')
activities = pd.read_csv('day1.csv')

for index, row in activities.iterrows():
    mask = (data['time'] >= row['start']) & (data['time'] <= row['end'])
    data.loc[mask, 'activity'] = row['activity']

data.to_csv('data/data_s_annotated.csv', index=False)